## MSCI 332 - Project Part 1

> **Team Members:** Maan Patel, Dhruv Hari, Nishesh Jagga, Edward Jeong <br>
> **Course:** MSCI 332 <br>
> **Topic:** Optimal EV chargers in a network <br>


# Model Definition

### Problem Definition 



Assumptions
- Each node will be a major intersection.
- If an optimal point for an EV charger is on an arc, we will move it to the adjacent node due to the P median problem. 
- 10% will be the minimum safety charge such that the user should start looking for a charger. 
- All charging stations are equipped with type 3 DC fast chargers that charge at the same speed. 
- Range increases linearly with battery energy level. 
- Total battery capacity and max range will be fixed. 
- We will also assume that there’s no waiting time at the station to avoid complexion with queues and a stochastic model (out of the scope of MSCI 332).
- We will assume that our network graph is one connected network where every vertex has some path to another.
The range of the car will be more than the distance between 2 adjacent nodes


## Mathematical Model

### **Sets**

$Ι: \text{Set of all intersections in the network } <br>
$A: \text{Set of all lengths of feasible links between intersections in the network } <br>
$R: \text{Set of all routes to traverse the network } <br>
$L: \text{Set of all distances from each intersection to the destination for all routes } <br>

### **Parameters**

$\tau: \text{Distance covered by EV using 90\% of range (100\% } \rightarrow \text{ 10\%)}$ <br>
$\beta: \text{Cost to build a charging station } <br>
$\alpha: \text{Cost to build a charger at a charging station } <br>
$a_{ij}: \text{ Length of link } (i,j) \in A$ <br>
$e_{ijk}: \begin{cases}  1 & \text{if EV travels from intersection } i \text{ to } j \text{ on link } (i,j) \text{ in route } k \\  0 & \text{otherwise}  \end{cases} \quad i,j \in I,\ (i,j) \in A,\ k \in R$ <br>
$f_{ik}: \text{Distance from intersection } i \in I \text{ to the destination in route } k \in R$ <br>
$b_{k}: \begin{cases}  f_{0k}-\tau & \text{if for route } k,\ f_{0k} \gt \tau  \\  0 & \text{otherwise}  \end{cases} \quad k \in R$ <br>

### **Decision Variables**

$x_{i}: \begin{cases}  1 & \text{if charging station exists at intersection } i  \\  0 & \text{otherwise}  \end{cases} \quad i \in I$ <br>
$y_{i}: \text{Number of chargers installed at intersection } i \in I$ <br>
$p_{ik}: \begin{cases}  1 & \text{if EV is charged at intersection } i \text{ on route } k  \\  0 & \text{otherwise}  \end{cases} \quad i \in I,\ k \in R$ <br>
$q_{ik}: \text{Range added by charging at intersection } i \in I \text{ on route } k \in R$ <br>
$r_{ik}: \text{Remaining range when arriving at intersection } i \in I \text{ on route } k \in R$ <br>

$$\text{MIN} \quad \sum\limits_{i \in I}{\beta x_{i} + \alpha y_{i}} $$

s.t.


# Implementation

## Creating/Visualizing the Network

In [ ]:
import numpy as np

In [ ]:
# Code taken from MSCI 332 Tutorial 2
# Random network generator
from itertools import combinations, groupby
import networkx as nx
import random
import matplotlib.pyplot as plt

np.random.seed(8953456)
NUMBER_OF_INTERSECTIONS = 8
RANGE_OF_CAR = 336
MIN_ARC_LENGTH = 80
MAX_ARC_LENGTH = 200


# original version from https://stackoverflow.com/a/61961881
def gnp_random_connected_graph(n, p):
    """
    Generates a random undirected graph, similarly to an Erdős-Rényi 
    graph, but enforcing that the resulting graph is conneted
    """
    edges = combinations(range(n), 2)
    G = nx.Graph()
    for i in range(n):
      G.add_node(i)

    if p <= 0:
        return G
    if p >= 1:
        return nx.complete_graph(n, create_using=G)
    for _, node_edges in groupby(edges, key=lambda x: x[0]):
      node_edges = list(node_edges)
      index = np.random.randint(len(node_edges))
      random_edge = node_edges[index]
      G.add_edge(random_edge[0], random_edge[1], weight=np.random.randint(MIN_ARC_LENGTH, MAX_ARC_LENGTH))
      for e in node_edges:
          if np.random.random() < p:
              G.add_edge(*e, weight=np.random.randint(MIN_ARC_LENGTH, MAX_ARC_LENGTH))
    return G

nodes = NUMBER_OF_INTERSECTIONS
probability = 0.25
G = gnp_random_connected_graph(nodes, probability)
labels = nx.get_edge_attributes(G,'weight')
pos = nx.spring_layout(G,k=5)
plt.figure(figsize=(11,9))
nx.draw_networkx_edge_labels(G,pos, edge_labels=labels)
nx.draw(G, pos, with_labels=True, node_color='lightblue',node_size=1000)
plt.show()


The **network** above will be used to determine the optimal locations for placements of EV chargers. The graph is a fully connected graph where every vertice has some path leading to another. Every node is an intersection. The arcs have a weight that falls in the range of a random number between **MIN_ARC_LENGTH** and **MAX_ARC_LENGTH**. The max arc length is much smaller than the **Range** of the car. This is realistic because in the real world, there are many intersections in the path. Usually the intersection is closer than the range of the car. This assumption also allows us to avoid placing the charger almost everywhere in the network. 

## Deterministic Model

In [ ]:
!pip install gurobipy>=9.5.1
import gurobipy as gp
from gurobipy import GRB as GRB
from gurobipy import quicksum

In [ ]:
# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', 'fcee3cac-af4b-4c4e-b808-7e384b08c4f8')
e.setParam('WLSSECRET', 'fdfef66d-12d2-4056-aff9-f3d03d038758')
e.setParam('LICENSEID', 868431)
e.start()

# Create the model within the Gurobi environment
model = gp.Model(env=e)

In [ ]:
import itertools

graph = nx.to_dict_of_dicts(G)
shortest_path = nx.dijkstra_path(G, 0, 2)

shortest_paths = nx.all_pairs_dijkstra_path(G)

arc_lengths = nx.get_edge_attributes(G,'weight')
arc_lengths.update({(k[1], k[0]) : v for k,v in arc_lengths.items()})

def get_arcs_from_nodes(node_list):
  return [(node_list[i-1], node_list[i]) for i in range (1, len(node_list))]

routes = []
ARC_ROWS = set()
for start_node in shortest_paths:
  for b in start_node[1].values():
    if len(b) > 1:
      arcs = get_arcs_from_nodes(b)
      arcs_w_lengths = {arc: arc_lengths[arc] for arc in arcs}
      arc_row = list(arcs_w_lengths.items())[0]
      ARC_ROWS.add(arc_row)
      routes.append({"nodes":tuple(b), "arcs":arcs_w_lengths})
print(len(ARC_ROWS))

In [ ]:
ARC_MATRIX = []
for i in range(NUMBER_OF_INTERSECTIONS):
  matrix_row = [0 for i in range(NUMBER_OF_INTERSECTIONS)]

  for row in ARC_ROWS:
    if row[0][0] == i:
      matrix_row[row[0][1]] = row[1]
  ARC_MATRIX.append(matrix_row)

In [ ]:
ROUTES = []
ROUTE_DISTANCES = []
ROUTE_ARCS = []
for j in routes:
  ROUTES.append((list(j["nodes"])))
  arc_lens = j['arcs']

  ROUTE_ARCS.append(list(arc_lens.keys()))

  arc_sum = sum(arc_lens.values())
  remaining_dist = []
  remaining_dist.append(arc_sum)
  for i in arc_lens.values():
    arc_sum -= i
    remaining_dist.append(arc_sum)
  ROUTE_DISTANCES.append(tuple(remaining_dist))

In [ ]:
ROUTE_ARC_BINARY = []

matrix = np.zeros((len(ROUTES), NUMBER_OF_INTERSECTIONS, NUMBER_OF_INTERSECTIONS), dtype=int)

for k in range(len(ROUTES)):
  arc = ROUTE_ARCS[k]

  for i in range(len(arc)):
    f = int(arc[i][0])
    s = int(arc[i][1])
    matrix[k][f][s] = 1

ROUTE_ARC_BINARY = matrix

In [ ]:
# Sets
I = list(G.nodes) # intersections in the network
A = ARC_MATRIX # lengths of feasible arcs/links between intersections
R = ROUTES # intersections/nodes of routes
L = ROUTE_DISTANCES # length of arcs/links of routes
F = ROUTE_ARCS # arcs/links of routes

In [ ]:
print(A)

In [ ]:
print(R)

In [ ]:
print(L)

In [ ]:
print(F)

In [ ]:
# Parameters
# Constants
TAU = int(RANGE_OF_CAR) # 90% of the Range of the car 
M = 10**9 # Big-M

# Indexed
a = np.array(A) # Length of link (i,j)
f = np.array(L, dtype=object) # Distance from intersection till destination
b = np.array([max(0, f[k][0] - TAU) for k, route in enumerate(L)]) # Additional range needed to reach destination
e = ROUTE_ARC_BINARY # If EV travels on link (i,j) in route k

In [ ]:
# model implementation
model = gp.Model("EV Charging Station Optimization Model")

# Decision Variables
x = model.addVars(I, vtype=GRB.BINARY, name="x") # Charging station exists at node
y = model.addVars(I, lb=0.0, vtype=GRB.INTEGER, name="y") # Num of chargers at node
p = model.addVars(I, range(len(R)), vtype=GRB.BINARY, name="p") # If EV is charged at node on a route
q = model.addVars(I, range(len(R)), lb=0.0, vtype=GRB.CONTINUOUS, name="q") # Range added to EV at node on a route
r = model.addVars(I, range(len(R)), lb=0.0, vtype=GRB.CONTINUOUS, name="r") # Remaining range when entering node on a route

In [ ]:
# Constraints
[model.addConstr(y[i] <= M * x[i]) for i in I] # 1
[model.addConstr(y[i] >= x[i]) for i in I] # 2
[model.addConstr(y[i] <= 10) for i in I] # max number of chargers at station
[model.addConstr(M * x[i] >= sum(p[i,k] for k, route in enumerate(R))) for i in I] # 3
[[model.addConstr(q[i,k] <= M * p[i,k]) for i in route] for k, route in enumerate(R)] # 4
[[model.addConstr(q[i,k] >= p[i,k]) for i in route] for k, route in enumerate(R)] # 5
[[model.addConstr(r[i,k] + q[i,k] <= TAU) for i in route] for k, route in enumerate(R)] # 6
[[model.addConstr(r[i,k] + q[i,k] == r[j,k] + a[i,j]) for (i,j) in arc] for k, arc in enumerate(F)] # 7
[model.addConstr(y[i] >= sum(p[i,k] for k, route in enumerate(R))) for i in I] # 8
[model.addConstr(sum(q[i,k] for i in I) == b[k]) for k, route in enumerate(R)] # 9
[model.addConstr(sum(p[i,k] for i in I) <= (b[k]/TAU) +1 ) for k, route in enumerate(R)] # 10


In [ ]:
# Fixed cost

charging_station_cost = sum((200_000 * x[i]) + (10_000 * y[i]) for i in I)
EV_route_cost = sum(x[0] for x in L)

# Objective
model.setObjective(charging_station_cost + EV_route_cost, sense=GRB.MINIMIZE)

## Result

In [ ]:
# optimize
model.optimize()
print(f"NUMBER OF CONSTRAINTS: {len(model.getConstrs())}")

In [ ]:
from networkx import exception
# output
try:
  print(f"\nOptimal cost: ${model.getObjective().getValue():,.0f}")
  color_map = []
  optimal_intersections = []
  for i in range(len(x)):
    # for j in y:
    if x[i].X and y[i].X:  # if variable is non-zero
      print(f"{int(y[i].X)} chargers will be place at intersection {i}")
      optimal_intersections.append(i)
  

  for node in G:
      if node in optimal_intersections:
          color_map.append('green')
      else: 
          color_map.append('blue')      

  # Draw Network
  plt.figure(figsize=(11,9))
  nx.draw_networkx_edge_labels(G,pos, edge_labels=labels)
  nx.draw(G, pos, with_labels=True, alpha=0.5, node_color=color_map,node_size=1000)
  plt.show()
  
except:
  print("Objective Solution Not Found")

As per the network graph above:

1. The green nodes are Charging Stations
2. The purple nodes are Intersections/Nodes